<a href="https://colab.research.google.com/github/byiringiroscar/NLP_FELLOWSHIP/blob/main/ScrapCharacterSelenium.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
%%shell
# Ubuntu no longer distributes chromium-browser outside of snap
#
# Proposed solution: https://askubuntu.com/questions/1204571/how-to-install-chromium-without-snap

# Add debian buster
cat > /etc/apt/sources.list.d/debian.list <<'EOF'
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster.gpg] http://deb.debian.org/debian buster main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster-updates.gpg] http://deb.debian.org/debian buster-updates main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-security-buster.gpg] http://deb.debian.org/debian-security buster/updates main
EOF

# Add keys
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A

apt-key export 77E11517 | gpg --dearmour -o /usr/share/keyrings/debian-buster.gpg
apt-key export 22F3D138 | gpg --dearmour -o /usr/share/keyrings/debian-buster-updates.gpg
apt-key export E562B32A | gpg --dearmour -o /usr/share/keyrings/debian-security-buster.gpg

# Prefer debian repo for chromium* packages only
# Note the double-blank lines between entries
cat > /etc/apt/preferences.d/chromium.pref << 'EOF'
Package: *
Pin: release a=eoan
Pin-Priority: 500


Package: *
Pin: origin "deb.debian.org"
Pin-Priority: 300


Package: chromium*
Pin: origin "deb.debian.org"
Pin-Priority: 700
EOF

# Install chromium and chromium-driver
apt-get update
apt-get install chromium chromium-driver

# Install selenium
pip install selenium

Executing: /tmp/apt-key-gpghome.njqIQn8QL9/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
gpg: key DCC9EFBF77E11517: public key "Debian Stable Release Key (10/buster) <debian-release@lists.debian.org>" imported
gpg: Total number processed: 1
gpg:               imported: 1
Executing: /tmp/apt-key-gpghome.c723Spywx3/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
gpg: key DC30D7C23CBBABEE: public key "Debian Archive Automatic Signing Key (10/buster) <ftpmaster@debian.org>" imported
gpg: Total number processed: 1
gpg:               imported: 1
Executing: /tmp/apt-key-gpghome.bTvcDSGeM0/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A
gpg: key 4DFAB270CAA96DFA: public key "Debian Security Archive Automatic Signing Key (10/buster) <ftpmaster@debian.org>" imported
gpg: Total number processed: 1
gpg:               imported: 1
Get:1 http://deb.debian.org/debian buster InRelease [122 kB]
Hit:2 http://archive.ubuntu.com/ubuntu 

In [3]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support import expected_conditions as etc
import json
import time
import os
import requests
from urllib.parse import urlparse, unquote
from bs4 import BeautifulSoup
from urllib.parse import urljoin
from selenium.webdriver.common.by import By
import pandas as pd

chrome_options = Options()


service = Service(executable_path=r'/usr/bin/chromedriver')
chrome_options.add_argument('--disable-blink-features=AutomationControlled')
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')


driver = webdriver.Chrome(service=service, options=chrome_options)

In [42]:
# # get individual page
def get_link_details(url):
  try:
    driver.get(url)
    try:
      composition = driver.find_element(By.ID, 'Composition')
      composition = composition.get_attribute("innerHTML")
    except:
      composition = ""
    try:
      description_head = driver.find_element(By.ID, 'ToneOfVoice')

        # Find all <li> elements inside the 'ToneOfVoice' element
      description_li = description_head.find_elements(By.TAG_NAME, 'li')

      # Iterate through <li> elements and print their text
      full_description = ""
      for element in description_li:
          full_description += element.get_attribute("innerHTML")
      soup = BeautifulSoup(full_description, 'html.parser')
      full_description = soup.get_text()
      full_description = full_description.strip()
    except:
      full_description = ""
    try:
      description_feature = driver.find_element(By.ID, 'ToneOfVoice')
      description_feature = description_feature.get_attribute('outerHTML')
      soup = BeautifulSoup(description_feature, 'html.parser')
      description_feature = soup.get_text()
      description_feature = description_feature.strip()

    except:
      description_feature = ""
    try:
      title_head = driver.find_element(By.TAG_NAME, 'h1')
      title_head = title_head.get_attribute("innerHTML")
      title_head = title_head.strip()
    except:
      title_head = ""

    try:
      images_links = []
      mydiv = driver.find_element(By.CLASS_NAME, 'ThumbNailNavClip')
      my_elements = mydiv.find_elements(By.TAG_NAME, 'a')
      for element in my_elements:
          images_links.append(element.get_attribute("rel"))
      image = images_links
    except:
      image = ""
    final_description = str(full_description) + " " + str(composition) + " " + str(description_feature)
    return image, title_head,  final_description
  except:
    pass


In [44]:
url = "https://www.next.co.uk/style/LS330923/K18761#K18761"
image, title_head,  final_description = get_link_details(url)
print(final_description)

 100% Polyester. This hooded dressing gown from Threadboys is perfect for keeping comfortable. Featuring a self-tie belt, reverse collar, two front pockets, and branded embroidered logo. Other styles and colours are also available.


In [ ]:
def get_links_scrapped(url):
  driver.get(url)
  items = []
  # Create a list to store the src attributes
  href_links = []

  last_height = driver.execute_script("return document.body.scrollHeight")
  itemTargetCount = 100

  while itemTargetCount > len(items):
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(1)
    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
      break
    last_height = new_height
    try:
      elements = driver.find_elements(By.CLASS_NAME, 'produc-1mup83m')
    except:
      pass
    for index, element in enumerate(elements):
        # image url from the pages list
          try:
            href = element.get_attribute("href")
            href_links.append({"category": url['category'], "link": href})
          except:
            pass
    # print(len(href_links))
  return href_links


In [ ]:
scrap_url = "https://www.next.co.uk/shop/promotion-charactershop"
url_scraped = get_links_scrapped(scrap_url)